In [12]:
import pandas as pd
from sqlalchemy import create_engine

# Store CSV into Dataframe 

In [36]:
eruptions = "Resources/eruptions.csv"
eruptions_data = pd.read_csv(eruptions)

events = "Resources/events.csv"
events_df = pd.read_csv(events)

volcano = "Resources/volcano.csv"
volcano_df = pd.read_csv(volcano)

damage = "Resources/damage.csv"
damage_df = pd.read_csv(damage)

